# **`cruw-devkit` Usages for ROD2021 Challenge**

This is a simple tutorial of `cruw-devkit` for the [ROD2021 Challenge](https://www.cruwdataset.org/rod2021). Please find the source code at https://github.com/yizhou-wang/cruw-devkit.

The tutorial is written by [Yizhou Wang](http://yizhouwang.net/).

## CRUW Dataset Object

The CRUW dataset object includes the sensor configurations, calibration parameters, object configurations, and coordinate mappings.

In [ ]:
from cruw import CRUW
data_root='/mnt/disk1/CRUW/ROD2021'

#### ROD2021 Dataset File Structure

Download [ROD2021 dataset](https://www.cruwdataset.org/download#h.mxc4upuvacso).
We put the data files as the following structure using [this script](https://github.com/yizhou-wang/cruw-devkit/tree/master/scripts/reorganize_rod2021.sh).

```
data_root
  - sequences
  | - train
  | | - <SEQ_NAME>
  | | | - IMAGES_0
  | | | | - <FRAME_ID>.jpg
  | | | | - ***.jpg
  | | | - RADAR_RA_H
  | | |   - <FRAME_ID>_<CHIRP_ID>.npy
  | | |   - ***.npy
  | | - ***
  | | 
  | - test
  |   - <SEQ_NAME>
  |   | - RADAR_RA_H
  |   |   - <FRAME_ID>_<CHIRP_ID>.npy
  |   |   - ***.npy
  |   - ***
  | 
  - annotations
  | - train
  | | - <SEQ_NAME>.txt
  | | - ***.txt
  | - test
  |   - <SEQ_NAME>.txt
  |   - ***.txt
  - calib
```

In [ ]:
dataset = CRUW(data_root=data_root, sensor_config_name='sensor_config_rod2021')
print(dataset)

### Camera and radar configurations

In [ ]:
print(dataset.sensor_cfg.camera_cfg)

In [ ]:
print(dataset.sensor_cfg.radar_cfg)

### Calibration parameters

In [ ]:
print('camera intrinsics:')
print(dataset.sensor_cfg.calib_cfg['cam_calib']['2019_04_09']['cam_0']['camera_matrix'])
print('camera to radar translation:')
print(dataset.sensor_cfg.calib_cfg['t_cl2rh'])

### Object classes of interest

In [ ]:
print(dataset.object_cfg.classes)

### Mapping between range/azimuth and the indices in RF images

In [ ]:
from cruw.mapping import ra2idx, idx2ra
import math

Map from absolute range (m) and azimuth (rad) to RF indices.

In [ ]:
rng = 5.0
azm = math.radians(30)  # degree to radians
rid, aid = ra2idx(rng, azm, dataset.range_grid, dataset.angle_grid)
print(rid, aid)

Map from RF indices to absolute range (m) and azimuth (rad).

In [ ]:
rid = 20
aid = 95
rng, azm = idx2ra(rid, aid, dataset.range_grid, dataset.angle_grid)
print(rng, math.degrees(azm))

Note: The range and azimuth cannot be absolutely recover due to the discretization from absolute range/azimuth values to RF pixels.

## Data Visualization

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from cruw.visualization.examples import show_dataset_rod2021

def get_paths(seq_name, frame_id):
    image_path = os.path.join(data_root, 'sequences', 'train', seq_name, 
                              dataset.sensor_cfg.camera_cfg['image_folder'], 
                              '%010d.jpg' % frame_id)
    chirp_path = os.path.join(data_root, 'sequences', 'train', seq_name, 
                              dataset.sensor_cfg.radar_cfg['chirp_folder'],
                              '%06d_0000.npy' % frame_id)
    anno_path = os.path.join(data_root, 'annotations', 'train', seq_name + '.txt')
    return image_path, chirp_path, anno_path

In [ ]:
seq_name = '2019_04_09_BMS1000'
frame_id = 400
image_path, chirp_path, anno_path = get_paths(seq_name, frame_id)
show_dataset_rod2021(image_path, chirp_path, anno_path, dataset)

In [ ]:
seq_name = '2019_04_09_PMS3000'
frame_id = 200
image_path, chirp_path, anno_path = get_paths(seq_name, frame_id)
show_dataset_rod2021(image_path, chirp_path, anno_path, dataset)

In [ ]:
seq_name = '2019_05_29_PBMS007'
frame_id = 300
image_path, chirp_path, anno_path = get_paths(seq_name, frame_id)
show_dataset_rod2021(image_path, chirp_path, anno_path, dataset)

In [ ]:
seq_name = '2019_09_29_ONRD001'
frame_id = 834
image_path, chirp_path, anno_path = get_paths(seq_name, frame_id)
show_dataset_rod2021(image_path, chirp_path, anno_path, dataset)

In [ ]:
seq_name = '2019_09_29_ONRD002'
frame_id = 900
image_path, chirp_path, anno_path = get_paths(seq_name, frame_id)
show_dataset_rod2021(image_path, chirp_path, anno_path, dataset)

In [ ]:
seq_name = '2019_09_29_ONRD011'
frame_id = 1000
image_path, chirp_path, anno_path = get_paths(seq_name, frame_id)
show_dataset_rod2021(image_path, chirp_path, anno_path, dataset)

## Evaluation Script

An example for the usage of evaluation tool used in the ROD2021 Challenge is shown below.

In [ ]:
from cruw.eval import evaluate_rod2021

submit_dir = '<SUBMISSION_DIR>'
truth_dir = '<ANNOTATION_DIR>'
evaluate_rod2021(submit_dir, truth_dir, dataset)